# Introduction

## About

This notebook gathers the code used for the miniproject "Bayesian Inverse Problems in Large Dimensions" of the Stochastic Simulation course. See the report for theoretical details.

## Setup

In [ ]:
# Google Colab compatibility
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd gdrive/MyDrive/mcmc-inverse-bayesian
except ModuleNotFoundError:
    pass

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard libraries
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from glob import glob
# Override matplotlib defaults for nicer plots
sns.set(style='whitegrid')

In [ ]:
# Custom imports
from utils import u, G, target_density
from mcmc import rwmh, diagnose_plot
from utils import dump_simulation_results, load_simulation_results

In [ ]:
# Given data
y = np.array([0.5041, 0.8505, 1.2257, 1.4113])

In [ ]:
# Noise standard deviation
sigma_noise = 0.04
# Chain length
N = int(1e4)

# Random Walk Metropolis

In this first approach, we use the proposal distribution

$$Q(\mathbf \theta, \cdot) = \mathcal N(\mathbf \theta, s^2 C), \quad C = \text{diag}(1, 2^{-2}, \ldots, D^{-2})$$

where $D$ is the dimension of the state space, being the number of Fourier coefficients used to approximate the log-permeability.

The goal is to generate chains of length $N=10^4$ with different values of $s$ and $D$. Remind that we're trying to probe the efficiency of MCMC for $D \to\infty$. We will use the `rwmh` (Random Walk Metropolis Hastings) utility function:

In [ ]:
help(rwmh)

In [ ]:
def simulate(args):
    """Perform one simulation, used as map function for multiprocessor pool"""
    D, s = args
    print(f'D = {D}, s = {s}\t', end='')
    X, p_accept = rwmh(
        ftilde=lambda theta: target_density(theta, sigma_noise, y),
        variances=s**2 / np.arange(1, D+1)**2,
        X0=np.zeros(D),
        N=N
    )
    # Only return the first component of the chain, too much data otherwise
    return {
        'X': X[:, 0],
        'D': D,
        's': s,
        'p_accept': p_accept
    }

In [ ]:
import multiprocessing as mp
import itertools

One can typically run the cell below in Google Colab with appropriate values for `s` and `D` and then import results with `load_simulation_results`:

In [ ]:
# D = (10, 100, 1000)
# s = (0.05, 0.1, 0.15, 0.2, 0.3, 0.5)
# pool = mp.Pool(mp.cpu_count())
# data = pool.map(simulate, itertools.product(D, s))
# pool.close()
# dump_simulation_results(data, 'data/rwmh_1.json.gz')

In [ ]:
df = pd.concat([
    pd.DataFrame(load_simulation_results(fpath)) for fpath in glob('data/rwmh_*.json.gz')
]).reset_index(drop=True).sort_values(['D', 's'])
df

In [ ]:
palette = sns.color_palette(n_colors=df.D.nunique())
sns.lineplot(x='s', y='p_accept', hue='D', data=df, marker='o', palette=palette)
plt.savefig('figs/1_rwmh_overview.pdf', bbox_inches='tight')

In [ ]:
df_diagnosis = df[(df.p_accept > 0.01)]
labels = [
    f'$D = {row.D}, s = {row.s}, p_a = {row.p_accept:.3}$' for _, row in df_diagnosis.iterrows()
]
diagnose_plot(df_diagnosis.X, labels, r'$\theta_1$')

In [ ]:
# data = []
# for D in (100,):
#     for s in (0.1, 0.3):
#         X, p_accept = rwmh(
#             ftilde=lambda theta: target_density(theta, sigma_noise, y),
#             variances=s**2 / np.arange(1, D+1)**2,
#             X0=np.zeros(D),
#             N=N
#         )
#         data.append({
#             'X': X,
#             'D': D,
#             's': s,
#             'p_accept': p_accept
#         })
#
# dump_simulation_results(data, 'data/rwmh_1.json.gz')

# Improvement: Preconditionned Crank-Nicholson

We modify the proposal distribution: the mean of the Gaussian is now biased toward zero:

$$q(\mathbf \theta, \cdot) = \mathcal N(\mathbf \theta \sqrt{1-s^2}, s^2 C)$$

In terms of the implementation, only a slight modification of `rwmh` is required: we just add a `precond_const` argument with value `1.0` by default (i.e., standard random walk metropolis), and we multiply the current state by this `precond_const` when shifting the pre-computed proposals.